Here is the solution for https://www.kaggle.com/c/titanic

In [86]:
import csv
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

train_column_names = ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', \
                'Age', 'SibSp', 'Parch', 'Ticket','Fare', 'Cabin', 'Embarked'] 
test_column_names = ['PassengerId', 'Pclass', 'Name', 'Sex', \
                'Age', 'SibSp', 'Parch', 'Ticket','Fare', 'Cabin', 'Embarked'] 

target = 'Survived'
features = ['Pclass', 'SexN', 'Age']

# load data from csv file
def read_data(file):
    with open(file, 'r', errors='backslashreplace') as f:
        reader = csv.DictReader(c.replace('\0', '') for c in f)
        for row in reader:
            yield row

def load_data(file):
    data = [d for d in read_data(file)]
    sex_numerical(data)
    return data

def print_data(data):
    for d in data:
        print(d)

def sex_numerical(data):
    for d in data:
        if d['Sex'] == 'male':
            d['SexN'] = 1
        else:
            d['SexN'] = 0

def numerical(val):
    try:
        return float(val)
    except ValueError:
        #return float('nan')
        return 0

def train(file):
    data = load_data(file)
    #print_data(data)

    X = [tuple(numerical(d[f]) for f in features) for d in data]
    #print_data(X)
    Y = [d[target] for d in data]
    #print_data(Y)
    print(len(X))
    print(len(Y))
    clf = LogisticRegression();
    clf.fit(X, Y);
    return clf
    

def test(file, clf):
    data = load_data(file)
    X = [tuple(numerical(d[f]) for f in features) for d in data]
    Y = [d[target] for d in data]
    score = clf.score(X, Y)
    return score

def predict(file, clf):
    data = load_data(file)
    X = [tuple(numerical(d[f]) for f in features) for d in data]
    score = clf.score(X)
    return score

In [87]:
train_file = 'train.csv'
test_file = 'test.csv'
output_file = 'output.csv'
clf = train(train_file)
score = test(train_file, clf)
print('score: {}'.format(score))


891
891
score: 0.7732884399551067
